In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [2]:
from pathlib import Path

In [3]:
dPath = Path("../docs/dumps")

In [4]:
import pickle

In [5]:
with open(dPath / "test_data.pkl", 'rb') as filename:
    test_data = pickle.load(filename)

In [6]:
with open(dPath / "mutclslabels.pkl", 'rb') as filename:
    mutclslabels = pickle.load(filename)
    
with open(dPath / "retypelabels.pkl", 'rb') as filename:
    retypelabels = pickle.load(filename)

In [7]:
with open(dPath / "rf_exp_05_names.pkl", 'rb') as filename:
    names = pickle.load(filename)

In [78]:
with open(dPath / "hgb_exp_02_stacking.pkl", 'rb') as filename:
    models = pickle.load(filename)

In [9]:
X_test = test_data.drop("Detected", axis=1)

In [10]:
y_test = test_data.Detected

In [11]:
X_test = X_test[names]

In [12]:
mutclscodes = dict(zip(mutclslabels,range(len(mutclslabels))))

In [13]:
retypecodes = dict(zip(retypelabels,range(len(retypelabels))))

In [14]:
X_test.replace(mutclscodes, inplace=True)

In [15]:
X_test.replace(retypecodes, inplace=True)

In [16]:
X_test.head()

,numExecuted,MutatorClass,numAssertInTC,numTestCover,numAssertInTM,ppavcc,cchalsteadCumulativeBugs,cchalsteadCumulativeVolume,ppRVF,ppnumberOfMethods,...,cchalsteadEffort,ccmaintainabilityIndexNC,mmhalsteadVocabulary,pptcc,mmexternalMethodsCalled,ppinstability,ppmaxcc,mmhalsteadEffort,mmvariablesReferenced,ccunweightedClassSize
0,32,5,2,16,2,1.258065,0.996047,2988.139644,16.0,31.0,...,31594.065211,117.568525,12.0,39.0,0.0,0.6,3.0,245.928428,1.0,12.0
1,1,0,2,1,1,1.258065,0.996047,2988.139644,16.0,31.0,...,31594.065211,117.568525,16.0,39.0,1.0,0.6,3.0,306.000000,1.0,12.0
2,1,4,2,1,1,1.258065,0.996047,2988.139644,16.0,31.0,...,31594.065211,117.568525,16.0,39.0,1.0,0.6,3.0,306.000000,1.0,12.0
3,1,0,2,1,1,1.258065,0.996047,2988.139644,16.0,31.0,...,31594.065211,117.568525,12.0,39.0,1.0,0.6,3.0,163.115794,0.0,12.0
4,1,4,2,1,1,1.258065,0.996047,2988.139644,16.0,31.0,...,31594.065211,117.568525,12.0,39.0,1.0,0.6,3.0,163.115794,0.0,12.0


In [79]:
predictions = []
for _model in models:
    predictions.append(_model.predict_proba(X_test)[:,1])
    
hgbs_preds_test = np.vstack(predictions).T

In [80]:
from scipy.stats.mstats import gmean

In [81]:
hgbs_pred_test = hgbs_preds_test.mean(axis=1)

In [82]:
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score, precision_score, recall_score
from sklearn import metrics

In [83]:
print(classification_report(y_test, hgbs_pred_test >=0.50))

              precision    recall  f1-score   support

       False       0.53      0.53      0.53     20633
        True       0.74      0.74      0.74     37843

    accuracy                           0.67     58476
   macro avg       0.64      0.64      0.64     58476
weighted avg       0.67      0.67      0.67     58476



In [84]:
print(average_precision_score(y_test, hgbs_pred_test >= 0.5, average='micro'))

0.719736286534185


In [85]:
metrics.roc_auc_score(y_test, hgbs_pred_test >= 0.5)

0.63810694481887

In [86]:
with open(dPath / "hgbs_exp_02_preds.pkl", 'wb') as filename:
    pickle.dump(hgbs_pred_test,filename)

In [87]:
hgbs_pred_test

array([0.59613989, 0.67995785, 0.66607214, ..., 0.40060204, 0.62277501,
       0.58070451])

In [91]:
np.std(hgbs_preds_test, axis=1)

array([0.03809655, 0.03988613, 0.04194271, ..., 0.04019107, 0.04404042,
       0.04473847])